In [1]:
import os
os.chdir("../")

In [2]:
from storage.har_datasets import STSDataset, StreamingTimeSeries, StreamingTimeSeriesCopy, HARTHDataset, split_by_test_subject
from storage.label_mappings import harth_label_mapping
from s3ts.dtw_layer.lightningstsds import LSTSDataset
from s3ts.dtw_layer.dtw_layer import DTWLayer
from s3ts.dtw_layer.wrapper import lmodel
from pytorch_lightning import Trainer
import numpy as np
import torch

In [3]:
ds = HARTHDataset("./datasets/HARTH/", wsize=80, normalize=True, label_mapping=harth_label_mapping)
print(len(ds))

6205301


In [4]:
data_split = split_by_test_subject(ds, 21)

dm = LSTSDataset(ds, data_split=data_split, batch_size=128, random_seed=42, num_workers=8)

In [5]:
model = torch.nn.Sequential(
    DTWLayer(8, 6, 48, l_out=48, rho=0.01),
    torch.nn.Conv2d(in_channels=8, out_channels=32, kernel_size=5), # out 44x44
    torch.nn.ReLU(),
    torch.nn.MaxPool2d(kernel_size=2), # out 22x22
    torch.nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5), # out 18x18
    torch.nn.ReLU(),
    torch.nn.MaxPool2d(kernel_size=2), # out 9x9
    torch.nn.Conv2d(in_channels=64, out_channels=128, kernel_size=4), # out 6x6 
    torch.nn.ReLU(),
    torch.nn.MaxPool2d(kernel_size=2), # out 3x3
    torch.nn.Flatten(),
    torch.nn.Linear(in_features=128*9, out_features=12)
)

In [6]:
trainer = Trainer(max_epochs=50, accelerator="auto")
model_lightning = lmodel(model, 12, lr=0.001)
trainer.fit(model=model_lightning, datamodule=dm)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name       | Type               | Params
--------------------------------------------------
0 | model      | Sequential         | 205 K 
1 | softmax    | Softmax            | 0     
2 | train_acc  | MulticlassAccuracy | 0     
3 | train_f1   | MulticlassF1Score  | 0     
4 | val_acc    | MulticlassAccuracy | 0     
5 | val_f1     | MulticlassF1Score  | 0     
6 | val_auroc  | MulticlassAUROC    | 0     
7 | test_acc   | MulticlassAccuracy | 0     
8 | test_f1    | MulticlassF1Score  | 0     
9 | test_auroc | MulticlassAUROC    | 0     
--------------------------------------------------
205 K     Trainable params
0         Non-trainable params
205 K     Total params
0.820     Total estimated model params size (MB)


Epoch 0:   0%|          | 30/47088 [00:11<5:00:33,  2.61it/s, v_num=54, train_loss_step=1.430]